## 2) Word-Level 번역기 만들기(Neural Machine Translation (seq2seq) Tutorial)

이전 챕터에서 seq2seq를 사용하여 글자 레벨(character-level)의 기계 번역기를 만들었다. 이번 챕터에서는 단어 레벨(Word-level)의 기계 번역기를 만들어보자. 모델 아키텍처는 이전 챕터와 거의 동일하지만, 단어 레벨을 수행하는 만큼 추가적인 전처리와 임베딩 층(Embedding layer), 그리고 추가적인 후처리 작업이 필요하다.

이번 챕터는 이전 챕터의 내용을 이해했다는 가정 하에 모델에 대한 설명을 자세하게 하지 않는다.


### 1.데이터 로드 및 전처리

필요한 도구들을 임포트한다.


In [1]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

데이터를 로드한다. 데이터는 이전 챕터와 동일한 데이터를 사용할 예정이다. 이에 데이터 구성에 대한 설명은 생략한다.


In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
ls

_about.txt  fra-eng.zip  fra.txt  sample_data/


이번 챕터에서는 총 33,000개의 샘플을 사용할 예정이다. 이 값을 변수에 지정한다.


In [4]:
num_samples = 33000

전처리 함수들을 구현한다.

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    

In [6]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만든다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환한다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

구현한 전처리 함수들을 임의의 문장을 입력으로 테스트해보자.


In [7]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent).encode('utf-8'))

have you had dinner ?
b'avez vous deja dine ?'


전체 데이터에서 33,000개의 샘플만 불러오되, 모든 전처리를 수행하는 함수를 만든다. 또한 훈련 과정에서 교사 강요(Teacher Forcing)을 사용할 예정이므로, 훈련 시 사용할 디코더의 입력 시퀀스와 실제값에 해당되는 출력 시퀀스를 따로 분리하여 저장한다. 입력 시퀀스에는 시작을 의미하는 토큰인 <(sos)>를 추가하고, 출력 시퀀스에는 종료를 의미하는 토큰인 <(eos)>를 추가한다.


In [8]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open('fra.txt', 'r') as lines:
        for i, line in enumerate(lines):

            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> " + tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

            if i == num_samples - 1:
                break
            
    return encoder_input, decoder_input, decoder_target

이렇게 얻은 3개의 데이터셋은 인코더의 입력, 디코더의 입력, 디코더의 실제값을 상위 5개 샘플만 출력해보자.


In [9]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


이제 케라스 토크나이저를 통해 단어 집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정을 거친다.


In [10]:
tokenizer_en = Tokenizer(filters = '', lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters = '', lower = False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

이어서 패딩을 수행한다.


In [11]:
encoder_input = pad_sequences(encoder_input, padding = 'post')
decoder_input = pad_sequences(decoder_input, padding = 'post')
decoder_target = pad_sequences(decoder_target, padding = 'post')

이렇게 얻은 데이터의 크기(shape)를 확인한다.


In [12]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 8)
(33000, 16)
(33000, 16)


샘플은 총 33,000개 존재하며 영어 문장의 길이는 8, 프랑스어 문장의 길이는 16이다. 단어 집합의 크기를 정의한다.


In [13]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print('영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}'.format(src_vocab_size, tar_vocab_size))


영어 단어 집합의 크기 : 4678, 프랑스어 단어 집합의 크기 : 8032


단어 집합의 크기는 각각 4,678개와 8,032개이다. 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어준다. 이들은 훈련을 마치고 예측 과정과 실제값과 결과를 비교하는 경우에 사용된다.


In [14]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

이제 테스트 데이터를 분리할 차례이다. 테스트 데이터를 분리하기 전에, 적절한 분포를 갖도록 데이터를 섞어주는 과정을 진행한다. 이를 위해서 우선 순서가 섞인 정수 시퀀스 리스트를 만든다.


In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[14723   408 10528 ... 21442  1069 12502]


이를 데이터셋의 순서로 지정해주면 샘플들이 기존 순서와 다른 순서로 섞이게 된다.


In [16]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

임의로 30,997번째 샘플을 출력해보자. 이때, decoder_input과 decoder_target은 데이터의 구조상으로 앞에 붙은 <(sos)> 토큰과 뒤에 붙은 <(eos)> 토큰을 제외하면 동일한 정수 시퀀스를 가져야하므로 이를 확인해주면 된다.


In [17]:
encoder_input[30997]

array([ 14,   8, 190,  17,  63,   1,   0,   0], dtype=int32)

In [18]:
decoder_input[30997]

array([  2,   9,   5, 943,  16, 274,   1,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

In [19]:
decoder_target[30997]

array([  9,   5, 943,  16, 274,   1,   3,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

저자의 경우 18, 5, 16, 173, 1이라는 동일 시퀀스를 확인했다. 이제 훈련 데이터의 10%를 테스트 데이터로 분리하겠다.



In [20]:
n_of_val = int(33000 * 0.1)
print(n_of_val)

3300


33,000개의 10%에 해당되는 3,300개의 데이터를 테스트 데이터로 사용한다. 


In [21]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

훈련 데이터와 테스트 데이터의 크기(shape)를 출력해보자.


In [22]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)

print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


훈련 데이터의 샘플은 29,700개, 테스트 데이터의 샘플은 3,300개가 존재한다. 이제 모델을 설계한다. 

### 2.기계 번역기 만들기

모델 설계를 위해 필요한 도구들을 임포트한다.


In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

임베딩 벡터와 LSTM의 은닉 상태의 크기를 특정 크기로 고정하고자 한다. 여기서는 50을 사용한다.


In [24]:
latent_dim = 50

인코더를 설계한다. Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할을 수행한다.



In [25]:
# 인코더
encoder_inputs = Input(shape = (None, ))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value = 0.0)(enc_emb) # 패딩 0은 연산에서 제외

encoder_lstm = LSTM(latent_dim, return_state = True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장


이제 디코더를 설계한다.


In [26]:
# 디코더
decoder_inputs = Input(shape = (None, ))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value = 0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences = True
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state = encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

모델의 입력과 출력을 정의함으로써 모델을 정의한다.


In [27]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

seq2seq의 디코더는 기본적으로 각각의 시점(timestep)에 대해서 다중 클래스 분류 문제를 풀고 있다. 매 시점마다 프랑스어 단어 집합의 크기의 선택지에서 단어를 1개 선택하여 이를 이번 시점에서 예측한 단어로 택한다. 다중 클래스 분류 문제이므로 위의 설계에서 출력층으로 소프트맥스 함수를 사용했다. 이 경우 손실 함수를 지금까지 categorical_crossentropy를 사용해왔다.

categorical_crossentropy를 사용하려면 레이블은 원-핫 인코딩이 된 상태여야 한다. 그런데 현재 decoder_outputs의 경우에는 원-핫 인코딩을 하지 않은 상태이다. 원-핫 인코딩을 하지 않은 상태로, 정수 레이블에 대해서 다중 클래스 분류 문제를 풀고자 하는 경우에는 categorical_crossentropy 함수가 아니라 sparse_categorical_crossentropy를 사용하면 된다. 이는 케라스에서 규정한 약속이다.


In [28]:
model.compile(optimizer = 'rmsprop',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])


모델의 파라미터를 확인해보자.


In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     233900      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     401600      input_2[0][0]                    
______________________________________________________________________________________________

현재 모델의 총 파라미터 개수는 1,082,972개이다. 이제 모델을 훈련한다. 128개의 배치 크기로 총 50 에포크 학습한다. 테스트 데이터를 검증 데이터로 사용하여 훈련이 제대로 되고 있는지 모니터링 하겠다.


In [30]:
hist = model.fit(x = [encoder_input_train, decoder_input_train], 
                 y = decoder_target_train,
                 validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
                 batch_size = 128,
                 epochs = 50)

Epoch 1/50
233/233 [==============================] - 23s 47ms/step - loss: 4.8117 - acc: 0.5889 - val_loss: 1.9102 - val_acc: 0.7046
Epoch 2/50
233/233 [==============================] - 9s 39ms/step - loss: 1.7848 - acc: 0.7229 - val_loss: 1.6320 - val_acc: 0.7391
Epoch 3/50
233/233 [==============================] - 9s 39ms/step - loss: 1.5582 - acc: 0.7461 - val_loss: 1.5340 - val_acc: 0.7505
Epoch 4/50
233/233 [==============================] - 9s 38ms/step - loss: 1.4569 - acc: 0.7611 - val_loss: 1.4327 - val_acc: 0.7717
Epoch 5/50
233/233 [==============================] - 9s 38ms/step - loss: 1.3612 - acc: 0.7804 - val_loss: 1.3671 - val_acc: 0.7830
Epoch 6/50
233/233 [==============================] - 9s 38ms/step - loss: 1.2855 - acc: 0.7900 - val_loss: 1.3097 - val_acc: 0.7927
Epoch 7/50
233/233 [==============================] - 9s 39ms/step - loss: 1.2305 - acc: 0.7974 - val_loss: 1.2626 - val_acc: 0.7971
Epoch 8/50
233/233 [==============================] - 9s 39ms/step -

저자의 경우 최종 에포크에서 훈련 데이터는 88%의 정확도를, 테스트 데이터에서는 85%의 정확도를 얻었다.



### 3.seq2seq 기계 번역기 동작시키기

seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 다르다. 그래서 테스트 과정을 위해 모델을 다시 설계해주어야 한다. 특히 디코더를 많이 수정해야 한다. 우선 테스트 과정에서의 인코더 모델을 설계한다.


In [31]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

디코더를 설계한다.


In [32]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape = (latent_dim, ))
decoder_state_input_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

디코더를 정의한다.


In [33]:
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs2] + decoder_states2)


테스트 과정을 위한 모델 설계를 완료하였다. 이제 테스트 과정에서의 동작을 위한 decode_sequence 함수를 구현한다.


In [37]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정한다.
    while not stop_condition:
        # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

결과 확인을 위한 함수를 만든다.

In [38]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp = ''
    for i in input_seq:
        if (i != 0):
            temp = temp + index_to_src[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp = ''
    for i in input_seq:
        if ((i != 0 and i != tar_to_index['<sos>']) and i != tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과를 출력해보자.


In [39]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('원문 : ', seq2src(encoder_input_train[seq_index]))
    print('번역문 :', seq2tar(decoder_input_train[seq_index]))
    print('예측문 :', decoded_sentence[:-5])
    print('\n')

원문 :  i told tom . 
번역문 : j en ai parle a tom . 
예측문 :  j ai dit que tom . 


원문 :  give me the ball . 
번역문 : donnez moi la balle ! 
예측문 :  donne moi la main ! 


원문 :  i saw tom naked . 
번역문 : j ai vu tom nu . 
예측문 :  j ai vu tom a vu . 


원문 :  you re satisfied . 
번역문 : vous etes content . 
예측문 :  vous etes heureux . 


원문 :  nice shot ! 
번역문 : joli coup ! 
예측문 :  mange tes ! 




https://medium.com/analytics-vidhya/neural-machine-translation-using-bahdanau-attention-mechanism-d496c9be30c3